## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-03-17-10-16 +0000,nypost,Jill Biden’s ex-husband charged with first-deg...,https://nypost.com/2026/02/03/us-news/jill-bid...
1,2026-02-03-17-08-05 +0000,bbc,BBC on the front line with Colombia's war on d...,https://www.bbc.com/news/videos/cp9jg1kr9pjo?a...
2,2026-02-03-17-05-06 +0000,bbc,Iran's president says it is ready to negotiate...,https://www.bbc.com/news/articles/c4gw10mkz0yo...
3,2026-02-03-17-03-00 +0000,startribune,RandBall: The Vikings’ narrative has undergone...,https://www.startribune.com/minnesota-vikings-...
4,2026-02-03-17-00-00 +0000,wsj,The Trump administration says its crackdown on...,https://www.wsj.com/politics/policy/do-more-de...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2480/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,44
44,epstein,28
8,new,16
116,ice,16
543,deal,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
91,2026-02-03-14-44-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...,112
281,2026-02-02-22-44-00 +0000,wsj,Trump demanded Monday that House lawmakers bac...,https://www.wsj.com/politics/policy/trump-push...,111
356,2026-02-02-18-39-33 +0000,nypost,"Trump says Jeffrey Epstein, Michael Wolff ‘con...",https://nypost.com/2026/02/02/us-news/trump-sa...,100
382,2026-02-02-17-14-00 +0000,wsj,President Trump’s choice of Kevin Warsh as cha...,https://www.wsj.com/economy/central-banking/wh...,90
250,2026-02-03-00-37-12 +0000,nyt,"Trump Announces Initial Trade Deal With India,...",https://www.nytimes.com/2026/02/02/us/politics...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
91,112,2026-02-03-14-44-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...
281,99,2026-02-02-22-44-00 +0000,wsj,Trump demanded Monday that House lawmakers bac...,https://www.wsj.com/politics/policy/trump-push...
29,61,2026-02-03-16-19-18 +0000,bbc,Son of Norway's crown princess hears details o...,https://www.bbc.com/news/articles/ckgn8dnxjgwo...
273,50,2026-02-02-22-57-52 +0000,cbc,U.S. judge grants asylum to Chinese whistleblo...,https://www.cbc.ca/radio/asithappens/china-whi...
107,48,2026-02-03-13-51-03 +0000,nypost,Disney names theme parks boss Josh D’Amaro as ...,https://nypost.com/2026/02/03/business/disney-...
382,44,2026-02-02-17-14-00 +0000,wsj,President Trump’s choice of Kevin Warsh as cha...,https://www.wsj.com/economy/central-banking/wh...
275,44,2026-02-02-22-51-30 +0000,cbc,Arizona police suspect Today show host's mothe...,https://www.cbc.ca/news/world/nancy-guthrie-mi...
267,40,2026-02-02-23-08-00 +0000,wsj,The Trump administration is preparing to devel...,https://www.wsj.com/politics/policy/trump-admi...
20,40,2026-02-03-16-33-53 +0000,nypost,Savannah Guthrie’s missing mom’s pacemaker sto...,https://nypost.com/2026/02/03/us-news/savannah...
279,39,2026-02-02-22-47-03 +0000,nypost,CBS News weighs firing Peter Attia in wake of ...,https://nypost.com/2026/02/02/media/cbs-news-w...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
